In [29]:
from langgraph.graph import StateGraph , START, END
from typing import TypedDict , Literal , Annotated
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage , SystemMessage

In [30]:
generator_llm = ChatOllama(
    model="gemma:2b",
)
evaluator_llm = ChatOllama(
    model = "gemma:2b"
)
optimizer_llm = ChatOllama(
    model = "gemma:2b"
)

In [31]:
from pydantic import BaseModel , Field

class TweetEvaluation(BaseModel):
    evaluation: Literal['approved','needs_improvement']= Field(..., description='Final evaluation result'),
    feedback: str = Field(..., description='Feedback for the tweet')

In [32]:
structured_evaluated_llm = evaluator_llm.with_structured_output(TweetEvaluation)

c:\Users\USER\Desktop\LangGraph\venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='Final evaluation result'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [33]:
class TweetState(TypedDict):
    
    topic : str 
    tweet : str
    evalution : Literal['approved','needs_improvement']
    feedback : str
    iteration : int
    max_iterations : int

In [34]:
def generate_tweet(state: TweetState) -> TweetState:
    
    messages = [
    SystemMessage(content="You are a funny and clever Twitter/X influencer."),
    HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
]

    response = generator_llm.invoke(messages).content
    
    return {'tweet': response}
    
    

In [35]:
def evaluate_tweet(state: TweetState) -> TweetState:
    
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality - Is this fresh, or have you seen it a hundred times before?
2. Humor - Did it genuinely make you smile, laugh, or chuckle?
3. Punchiness - Is it short, sharp, and scroll-stopping?
4. Virality Potential - Would people retweet or share it?
5. Format - Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Don't end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"
- feedback: One paragraph explaining the strengths and weaknesses
""")
]
    
    response = structured_evaluated_llm.invoke(messages)

    
    return {
    'evaluation': response['evaluation'],
    'feedback': response['feedback']
    }



In [36]:
def optimize_tweet(state: TweetState):
    
    messages = [
    SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
    HumanMessage(content=f"""Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
]
    
    response = optimizer_llm(messages).content
    iteration = state['iteration'] + 1

    return {'tweet': response , 'iteration': iteration}

In [37]:
def iterate_evaluation(state: TweetState):
    
    if state['evalution'] == 'approved' or state['iteration'] >= state['max_iterations']:
        return 'approved'
    else :
        return 'needs_improvement'

In [38]:
graph = StateGraph(TweetState)

graph.add_node('generate',generate_tweet)
graph.add_node('evaluate',evaluate_tweet)
graph.add_node('optimize',optimize_tweet)

graph.add_edge(START , 'generate')
graph.add_edge('generate', 'evaluate')

graph.add_conditional_edges('evaluate', iterate_evaluation, { 'approved' : END , 'needs_improvement' : 'optimize' })

graph.add_edge('optimize' , 'evaluate')

workflow = graph.compile()

In [39]:
initial_state = {
    'topic' : 'World Peace',
    'iteration' : 1,
    'max_iterations' : 5,
}

workflow.invoke(initial_state)

c:\Users\USER\Desktop\LangGraph\venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='Final evaluation result'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


TypeError: 'TweetEvaluation' object is not subscriptable